In [ ]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.utils.validation import check_random_state
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
class KMeans(BaseEstimator, ClusterMixin):
    def __init__(self, n_clusters=8, max_iter=300, tol=1e-4, random_state=None):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state

    def fit(self, X):
        random_state = check_random_state(self.random_state)
        n_samples, n_features = X.shape

        # Initialize cluster centers randomly
        self.cluster_centers_ = X[random_state.choice(n_samples, self.n_clusters, replace=False)]

        for i in range(self.max_iter):
            # Assign labels based on closest center
            labels = self._assign_labels(X)

            # Compute new centers as the mean of the points in each cluster
            new_centers = np.array([X[labels == j].mean(axis=0) for j in range(self.n_clusters)])

            # Check for convergence
            if np.linalg.norm(new_centers - self.cluster_centers_) < self.tol:
                break

            self.cluster_centers_ = new_centers

        self.labels_ = labels
        return self

    def _assign_labels(self, X):
        # Calculate distances from points to cluster centers
        distances = euclidean_distances(X, self.cluster_centers_)
        # Assign each point to the nearest cluster center
        return np.argmin(distances, axis=1)

    def fit_predict(self, X):
        # Fit the model and return cluster labels for the dataset
        self.fit(X)
        return self.labels_

    def split_clusters(self, X, is_recursive=False, split_clusters = None):
        if is_recursive:
            unique_clusters = split_clusters
        else:
            # For initial call, process all clusters
            unique_clusters = np.unique(self.labels_)

        for i in unique_clusters:
            # Get data points belonging to the current cluster
            cluster_data = X[self.labels_ == i]
            filtered_cluster_data = self.remove_edge_points(cluster_data, i)
            density_matrix = self.get_density_matrix(filtered_cluster_data)

            if self.is_chain_connected(density_matrix):
                # Run K-Means to split the cluster into two
                kmeans_split = KMeans(n_clusters=2, random_state=self.random_state)
                split_labels = kmeans_split.fit_predict(cluster_data)

                # Assign new cluster numbers
                max_label = self.labels_.max() + 1  # New cluster number
                for idx, point in enumerate(cluster_data):
                    original_index = np.where((X == point).all(axis=1))[0][0]
                    if split_labels[idx] == 1:
                        self.labels_[original_index] = max_label  # Assign new cluster label

                # Update the cluster centers
                self.cluster_centers_[i] = kmeans_split.cluster_centers_[0]  # Update existing cluster center
                self.cluster_centers_ = np.vstack([
                    self.cluster_centers_,
                    kmeans_split.cluster_centers_[1]  # Add new cluster center
                ])

                # Increment cluster count
                self.n_clusters += 1
                self.labels_ = self.split_clusters(X, is_recursive=True, split_clusters = [i, max_label])


        return self.labels_


    def remove_edge_points(self, cluster_data, i):
        distances = euclidean_distances(cluster_data, self.cluster_centers_[i].reshape(1, -1)).flatten()
        threshold = distances.mean() + 1.5 * distances.std()
        filtered_cluster_data = cluster_data[distances <= threshold]
        return filtered_cluster_data

    def is_chain_connected(self, matrix):
        rows, cols = len(matrix), len(matrix[0])

        def trim_edges(matrix):
            """Trims 15% of rows and columns from all sides of the matrix."""
            start_row, end_row = round(rows * 0.15), round(rows * 0.85)
            start_col, end_col = round(cols * 0.15), round(cols * 0.85)
            trimmed = [row[start_col:end_col] for row in matrix[start_row:end_row]]
            return trimmed

        def bfs(matrix, start_row, start_col, target_row=None, target_col=None):
            """Breadth-first search to find a valid path to the target."""
            queue = [(start_row, start_col)]
            visited = set(queue)
            rows, cols = len(matrix), len(matrix[0])

            while queue:
                x, y = queue.pop(0)

                # Check if we've reached the target
                if (target_row is not None and x == target_row) or (target_col is not None and y == target_col):
                    return True

                # Explore neighbors
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < rows and 0 <= ny < cols and matrix[nx][ny] == 0 and (nx, ny) not in visited:
                        visited.add((nx, ny))
                        queue.append((nx, ny))

            return False

        # Trim the matrix
        trimmed_matrix = trim_edges(matrix)

        # Check for a top-to-bottom chain
        for col in range(len(trimmed_matrix[0])):
            if trimmed_matrix[0][col] == 0:
                if bfs(trimmed_matrix, 0, col, target_row=len(trimmed_matrix) - 1):
                    print("Top to bottom chain found")
                    return True

        # Check for a left-to-right chain
        for row in range(len(trimmed_matrix)):
            if trimmed_matrix[row][0] == 0:
                if bfs(trimmed_matrix, row, 0, target_col=len(trimmed_matrix[0]) - 1):
                    print("Left to right chain found")
                    return True

        return False


    def get_density_matrix(self, filtered_cluster_data):
        # Calculate x_min, x_max, y_min, y_max
        x_min = filtered_cluster_data[:, 0].min()
        x_max = filtered_cluster_data[:, 0].max()
        y_min = filtered_cluster_data[:, 1].min()
        y_max = filtered_cluster_data[:, 1].max()
        square_size = 0.05 * min(x_max - x_min, y_max - y_min)
        total_points = len(filtered_cluster_data)
        rectangle_area = (x_max - x_min) * (y_max - y_min)
        average_density = total_points / rectangle_area

        # Set threshold as a fraction of the average density
        density_threshold = 0.5 * average_density * (square_size**2)

        x_coords = np.arange(x_min, x_max, square_size)
        y_coords = np.arange(y_min, y_max, square_size)

        rows = len(x_coords)
        cols = len(y_coords)

        # Initialize the density matrix with 1s
        density_matrix = np.ones((rows, cols), dtype=int)

        # Populate the density matrix
        for i, x in enumerate(x_coords):
            for j, y in enumerate(y_coords):
                # Define square boundaries
                square = {
                    "x_min": x,
                    "x_max": x + square_size,
                    "y_min": y,
                    "y_max": y + square_size,
                }

                # Count points within the square
                points_in_square = filtered_cluster_data[
                    (filtered_cluster_data[:, 0] >= square["x_min"]) &
                    (filtered_cluster_data[:, 0] < square["x_max"]) &
                    (filtered_cluster_data[:, 1] >= square["y_min"]) &
                    (filtered_cluster_data[:, 1] < square["y_max"])
                ]

                # Check if density is below the threshold
                density = len(points_in_square)
                if density < density_threshold:
                    density_matrix[i, j] = 0

        return density_matrix